<html>
<head>
  <title>AUTOMAÇÃO com SELENIUN</title>
</head>
<body>
    <p>Projeto: <b>BrTaskerBot</b></p>
  <h1>FUNÇÃO 01</h1>
  <h2>Relatório de Inadimplência</h2>
  </body>
</html>

In [78]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm
import pyautogui as py
import pandas as pd
import time
import os

servico = Service(ChromeDriverManager().install())
url = "https://ssl.brcondos.com.br/Reports/UnitOverdueSynthetic"

## Funções Scrapping

In [79]:
def scrapp_resumo(condo):
    # Pegar RESUMO da Indimplência
    tabela_resumo_inadimp = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[2]/div/table/tbody')))
    tabela_resumo_inadimp = tabela_resumo_inadimp.text
    # Processar o conteúdo de texto para criar uma lista de linhas
    linhas_resumo = tabela_resumo_inadimp.split('\n')
    # Processar cada linha para criar uma lista de colunas
    dados_resumo = [linha.split() for linha in linhas_resumo]
    # Criar o DataFrame
    df_resumo_inadimp = pd.DataFrame(dados_resumo)
    display(df_resumo_inadimp)

In [80]:
def scrapp_relatorio(condo):
    # Pegar RELATÓRIO da Indimplência
    tabela_relatorio_inadimp = wait.until(EC.presence_of_element_located((
        By.XPATH, '//*[@id="toLoading"]/div/div/div')))
    tabela_relatorio_inadimp = tabela_relatorio_inadimp.text
    # Processar o conteúdo de texto para criar uma lista de linhas
    linhas_relatorio = tabela_relatorio_inadimp.split('\n')
    # Processar cada linha para criar uma lista de colunas
    dados_relatorio_inadimp = [linha.split() for linha in linhas_relatorio]
    # Criar o DataFrame
    df_relatorio_inadimp = pd.DataFrame(dados_relatorio_inadimp)
    # Exibir o DataFrame
    display(df_relatorio_inadimp.head(5))

## Dados

In [81]:
# Database dos condomínio
tabela = pd.read_excel(r"TABELACONDOMINIOS.xlsx", sheet_name="JUR")
condos = tabela["CONDOMINIO"].tolist()
adv = tabela["ADVOGADO"].tolist()
emailadv = tabela["EMAILADVOGADO"].tolist()
sindico = tabela["SINDICO"].tolist()
emailsindico = tabela["EMAILSINDICO"].tolist()
#display(condo)

## Navegador

In [82]:
# Configurações do Navegador // Desativar as Notificações do Chrome
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)
# chrome_options.add_argument("--headless")  # Executar em modo headless

# Navegador e tempo de espera padrão
navegador = webdriver.Chrome(service=servico, options=chrome_options)
wait = WebDriverWait(navegador, 2)

In [83]:
# logar no sistema BRCondos
navegador.get(url)

navegador.find_element('xpath', '//*[@id="Login"]').send_keys(input("Digite o seu email: "))
navegador.find_element('xpath', '//*[@id="Password"]').send_keys(input("Digite sua senha: "))
navegador.find_element('xpath', '//*[@id="form_login"]/button').click()

### Acesso inicial

In [84]:
## SELECIONA O CONDOMÍNIO INICIAL
time.sleep(2)
# Localizar e clicar no elemento para abrir o campo de entrada
navegador.find_element('xpath', '//*[@id="openModalSelect_chzn"]/div/div').click()

# Aguardar o campo de entrada e inserir o texto
campo_input_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn"]/div/div/input')))
campo_input_condominio.send_keys("ALEGRIA")

# Selecionar o condomínio do primeiro resultado
campo_select_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn_o_1"]')))
time.sleep(1)
campo_select_condominio.click()

time.sleep(1)

# Botão Filtrar por UNIDADE
navegador.find_element(By.XPATH, '//*[@id="form_post"]/div[1]/div[2]/div[2]/label[2]/span').click()

time.sleep(1)

# Gerar Relatório
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="form_post"]/div[3]/div')))
navegador.find_element(By.XPATH, '//*[@id="form_post"]/div[3]/div').click()

# Esperar tabela carregar
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="toLoading"]/div/div/div')))

time.sleep(1)

### Loop com Tratamento de erro

In [74]:
checkpoint_file = "checkpoint.txt"
max_errors = 3

# Verificar o último checkpoint
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        last_successful_index = int(f.read())
else:
    last_successful_index = 0

# Contador de erros
error_count = 0

# Loop sobre os condomínios
for i in tqdm(range(last_successful_index, len(condos))):
    condo = condos[i]
    
    while True:
        try:
            # Abrir janela para trocar condomínio
            navegador.find_element('xpath', '/html/body/header/div/div[3]/div/div[4]/div[1]/div/div[2]/div').click()
            time.sleep(1)
            
            # Localizar e clicar no elemento para abrir o campo de entrada
            elemento_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn"]/div/div')))
            time.sleep(2)
            elemento_condominio.click()

            # Localizar o campo de entrada e inserir o texto
            campo_input_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn"]/div/div/input')))
            time.sleep(2)
            campo_input_condominio.send_keys(condo)

            # Selecionar o condomínio do primeiro resultado
            campo_select_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn_o_1"]')))
            time.sleep(1)
            campo_select_condominio.click()

            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="toLoading"]/div/div/div/table/tbody')))

            time.sleep(1)
            
            
            #### FUNÇÕES SCRAPPING ####

            print("*" * 30, condo, "*" * 30)
            print( )
            print ("RESUMO DA INADIMPLÊNCIA")
            scrapp_resumo(condo)

#             print("*" * 80)
#             print( )
#             print ("RELATÓRIO")
#             scrapp_relatorio(condo)

            ###########################

            
            # Atualizar o checkpoint para o próximo índice
            with open(checkpoint_file, "w") as f:
                f.write(str(i + 1))
            
            # Redefinir o contador de erros
            error_count = 0
            
            break
        except WebDriverException as e:
            print(f"Erro ao processar o condomínio {condo}: {str(e)}")
            if "502 Bad Gateway" in str(e):
                print("Tentando novamente após atualizar a página...")
                navegador.refresh()
                time.sleep(2)
            else:
                error_count += 1
                print(f"Tentativa de número {error_count}")
                if error_count >= max_errors:
                    print("Muitos erros consecutivos. Atualizando e retomando...")
                    with open(checkpoint_file, "w") as f:
                        f.write(str(i))
                    navegador.refresh()
                    time.sleep(2)
                    error_count = 0
                else:
                    print("Tentando novamente...")
                    time.sleep(2)

  0%|                                                                                           | 0/40 [00:00<?, ?it/s]

****************************** ALDEIA DA MATA ******************************

RESUMO DA INADIMPLÊNCIA


,0,1,2,3,4
0,Data,Base,29/06/2023,None,None
1,Unidades,Inadimplentes,07,"(4,86%)",None
2,Valor,Total,R$,"60.802,16",None
3,Valor,Total,Juros,R$,"45.835,57"
4,Valor,Total,Multa,R$,"1.623,43"
5,Valor,Total,Correção,R$,"20.371,59"
6,Valor,Total,Devido,R$,"128.632,75"
7,Quantidade,de,Faturas,134,None


  2%|██                                                                                 | 1/40 [00:14<09:44, 14.98s/it]

****************************** ALEGRIA ******************************

RESUMO DA INADIMPLÊNCIA


...

,0,1,2,3,4
0,Data,Base,29/06/2023,None,None
1,Unidades,Inadimplentes,43,"(11,94%)",None
2,Valor,Total,R$,"111.869,88",None
3,Valor,Total,Juros,R$,"58.878,04"
4,Valor,Total,Multa,R$,"2.814,50"
5,Valor,Total,Correção,R$,"28.850,39"
6,Valor,Total,Devido,R$,"202.412,81"
7,Quantidade,de,Faturas,348,None


 98%|███████████████████████████████████████████████████████████████████████████████▉  | 39/40 [12:49<00:19, 19.35s/it]

****************************** VIVA RESIDENCIAL ******************************

RESUMO DA INADIMPLÊNCIA


,0,1,2,3,4
0,Data,Base,29/06/2023,None,None
1,Unidades,Inadimplentes,80,"(15,87%)",None
2,Valor,Total,R$,"167.604,10",None
3,Valor,Total,Juros,R$,"20.191,61"
4,Valor,Total,Multa,R$,"3.390,16"
5,Valor,Total,Correção,R$,"1.884,71"
6,Valor,Total,Devido,R$,"193.070,58"
7,Quantidade,de,Faturas,549,None


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [13:11<00:00, 19.78s/it]


In [62]:
# Limpar histórico do loop
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)

## Excel

In [66]:
df_relatorio_inadimp.to_excel(f'BRCondos Relatorio de Inadimplência ALEGRIA ddmmaaaa.xlsx.xlsx', index=False)

In [75]:
df_resumo_inadimp.to_excel(f'BRCondos Resumo da Inadimplência ALL CONDOS.xlsx', index=False)

# Loop

In [135]:
for condo in tqdm(condos):
    
    # Abrir janela para trocar condomínio
    navegador.find_element('xpath', '/html/body/header/div/div[3]/div/div[4]/div[1]/div/div[2]/div').click()
    time.sleep(1)
    
    # Localizar e clicar no elemento para abrir o campo de entrada
    elemento_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn"]/div/div')))
    time.sleep(2)
    elemento_condominio.click()

    # Localizar o campo de entrada e inserir o texto
    campo_input_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn"]/div/div/input')))
    time.sleep(1)
    campo_input_condominio.send_keys(condo)

    # Selecionar o condomínio do primeiro resultado
    campo_select_condominio = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="openModalSelect_chzn_o_1"]')))
    time.sleep(1)
    campo_select_condominio.click()

    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="toLoading"]/div/div/div')))

    time.sleep(1)

## Scrapping

In [85]:
# Pegar RESUMO da Indimplência
tabela_resumo_inadimp = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[2]/div/table/tbody')))
tabela_resumo_inadimp = tabela_resumo_inadimp.text
# Processar o conteúdo de texto para criar uma lista de linhas
linhas_resumo = tabela_resumo_inadimp.split('\n')
# Processar cada linha para criar uma lista de colunas
dados_resumo = [linha.split() for linha in linhas_resumo]
# Criar o DataFrame
df_resumo_inadimp = pd.DataFrame(dados_resumo)
# Exibir o DataFrame
print("*"*30, "ALEGRIA", "*"*30)
print(df_resumo_inadimp)

****************************** ALEGRIA ******************************
            0              1           2           3           4
0        Data           Base  29/06/2023        None        None
1    Unidades  Inadimplentes          28    (28,00%)        None
2       Valor          Total          R$  120.765,66        None
3       Valor          Total       Juros          R$   38.204,34
4       Valor          Total       Multa          R$    2.801,51
5       Valor          Total    Correção          R$   19.328,66
6       Valor          Total      Devido          R$  181.100,17
7  Quantidade             de     Faturas         359        None


In [86]:
df_resumo_inadimp.to_excel(f'BRCondos Resumo da Inadimplência ALEGRIA.xlsx', index=False)

In [92]:
# Criar um novo DataFrame com as linhas e colunas desejadas
new_df_resumo_inadimp = pd.DataFrame({
    'Condominio': [condo],
    'Data base': [df_resumo_inadimp.loc[0, 2]],
    'Valor total': [df_resumo_inadimp.loc[6, 4]],
    'Inadimplentes': [df_resumo_inadimp.loc[1, 3]],
})

# Exibir o novo DataFrame
print(new_df_resumo_inadimp)

         Condominio   Data base Valor total Inadimplentes
0  VIVA RESIDENCIAL  29/06/2023  181.100,17      (28,00%)


In [42]:
# Pegar RELATÓRIO da Indimplência
tabela_relatorio_inadimp = wait.until(EC.presence_of_element_located((
    By.XPATH, '//*[@id="toLoading"]/div/div/div')))
tabela_relatorio_inadimp = tabela_relatorio_inadimp.text
# Processar o conteúdo de texto para criar uma lista de linhas
linhas_relatorio = tabela_relatorio_inadimp.split('\n')
# Processar cada linha para criar uma lista de colunas
dados_relatorio_inadimp = [linha.split() for linha in linhas_relatorio]
# Criar o DataFrame
df_relatorio_inadimp = pd.DataFrame(dados_relatorio_inadimp)
# Exibir o DataFrame
print(df_relatorio_inadimp.head(5))

           0     1     2           3      4      5      6         7       8   \
0           #  None  None        None   None   None   None      None    None   
1   Condômino  None  None        None   None   None   None      None    None   
2  Observação   Mês  Ref.  Vencimento  Valor  Juros  Multa  Correção  Devido   
3       BLOCO   01,    N°           1   (01)   None   None      None    None   
4     8652033  None  None        None   None   None   None      None    None   

     9   ...    59    60    61    62    63    64    65    66    67    68  
0  None  ...  None  None  None  None  None  None  None  None  None  None  
1  None  ...  None  None  None  None  None  None  None  None  None  None  
2  None  ...  None  None  None  None  None  None  None  None  None  None  
3  None  ...  None  None  None  None  None  None  None  None  None  None  
4  None  ...  None  None  None  None  None  None  None  None  None  None  

[5 rows x 69 columns]


In [60]:
#### FUNÇÕES SCRAPPING ####

# for condo in range(1):

print("*" * 30, condo, "*" * 30)
print( )
print ("RESUMO DA INADIMPLÊNCIA")
scrapp_resumo(condo)

print("*" * 80)
print( )
print ("RELATÓRIO")
scrapp_relatorio(condo)

###########################

****************************** 0 ******************************

RESUMO DA INADIMPLÊNCIA


,0,1,2,3,4
0,Data,Base,29/06/2023,None,None
1,Unidades,Inadimplentes,28,"(28,00%)",None
2,Valor,Total,R$,"120.765,66",None
3,Valor,Total,Juros,R$,"38.204,34"
4,Valor,Total,Multa,R$,"2.801,51"
5,Valor,Total,Correção,R$,"19.328,66"
6,Valor,Total,Devido,R$,"181.100,17"
7,Quantidade,de,Faturas,359,None


********************************************************************************

RELATÓRIO


,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,#,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Condômino,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Observação,Mês,Ref.,Vencimento,Valor,Juros,Multa,Correção,Devido,None,...,None,None,None,None,None,None,None,None,None,None
3,BLOCO,"01,",N°,1,(01),None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,8652033,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
